In [185]:
import camelot
import pandas as pd
import tabula
import re
import numpy as np

In [2]:
"""
takes title of table from first index of header and replaces headers with integers
"""
def replaceHeader(curr_head, df):
    df.iloc[0, 0] = curr_head[0]
    df.columns = range(df.shape[1])
    return df

In [149]:
"""
combines top 2 rows
"""
def topCombine(table):
    header = table.iloc[0,0]
    table = table.drop(0, axis = 0).reset_index(drop=True)
    table.iloc[0,0]= header
    return table

In [292]:
"""
cleans data when multiple values are batched into the same column
"""
def cleanOverlap(table, column = 0, year_row = 0, text_issue = True, financial = False, drop = False):
    
    
    # get number of columns that are wrong by search for years
    column_err = re.findall("20\d\d-?", table.iloc[year_row,column])

    # create lists of similar size
    lists = []
    
    #get the same num nan as starting year_row
    if text_issue == False:
        for nan in range(year_row):
            lists.append(["" for i in range(len(column_err))])
        
    for i in range(year_row, len(table)): 
        # get errors in column names up to 999,999,999
        numbers = re.findall("-?\d*[,.]?\d*[,.]?\d+%?-?",table.iloc[i,column])

        # make sure they match
        try:
            assert len(column_err) == len(numbers)
        except:
            print((column_err, numbers))
            numbers = numbers[1:]
            
        # remove number from original
        num = " ".join(numbers)
        text = re.sub(num, "", table.iloc[i,column])
        if text_issue:
            lists.append([text] + numbers)
        else:
            lists.append(numbers)
    lists = pd.DataFrame(lists)
    
    if financial:
        table = table.dropna(axis = 0, thresh = 3).reset_index(drop = True)
        
    # append columns back into array in the right order
    table = table.dropna(axis = 1, thresh = 7)
    
    if drop:
        table = table.drop(0, axis = 0).reset_index(drop=True)
        
    if text_issue:
        table = pd.concat([lists, table.iloc[:,1:]], axis = 1) # how to insert new columns in the middle
    else:
        table = pd.concat([table.iloc[:,:column], lists, table.iloc[:,(column + 1):]], axis = 1) # how to insert new columns in the middle
    return table

In [819]:
"""
matches numbers in the text and moves them to the right column
"""
def fillBlanks(table, year_row = 0, columns = 0):
        
    for i in range(year_row, len(table)): 
        # get errors in column names up to 999,999,999
        try:
            numbers = re.findall("-?\d*[,.]?\d*[,.]?\d+%?-?",table.iloc[i,columns])

            if numbers:
                try:
                    text = re.sub(numbers[0], "", table.iloc[i,columns])
                    table.iloc[i,columns] = text
                    table.iloc[i,columns+1] = numbers[0]
                except:
                    print(numbers, table.iloc[i,columns])
        except:
            print(i)
            
    return table

In [750]:
#tables = camelot.read_pdf('Energy-Revolution-2015-Full.pdf')

left = 21.97
middle = 21+ 274.96
right = 540

ex_page = 54
final_page = 55

config = 7

table1 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (53.02, 21.97, 53.02 + 344.91, 21+ 274.96), columns = [21+274.96])
t1head = table1.columns


#14, 18, 22, 26, 30, 34, 3
if config ==0:
    table1 = table1.iloc[:, :7]
    table1 = replaceHeader(t1head, table1)
if config == 1:
    table1 = table1.iloc[:, :6]
    table1 = cleanOverlap(table1, drop =True)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-1]
if config == 2:
    table1 = table1.iloc[:, :7]
    table1 = cleanOverlap(table1, drop =False)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-1]
if config == 3:
    table1 = table1.iloc[:, :8]
    #table1 = cleanOverlap(table1, drop =False)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-1]
if config == 4:
    table1 = table1.iloc[:, :8]
    #table1 = cleanOverlap(table1, drop =False)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-2]
if config == 5:
    table1 = table1.iloc[:, :8]
    table1 = cleanOverlap(table1, drop =False)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-2]
if config == 6:
    table1 = table1.iloc[:, :7]
    table1 = cleanOverlap(table1, drop =False)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-2]
if config == 7:
    table1 = table1.iloc[:, :8]
    table1 = cleanOverlap(table1, drop =False)
    table1 = replaceHeader(t1head, table1)
    table1 = table1.iloc[:,:-2]
table1

(['2012', '2020'], ['2', '0', '0'])
(['2012', '2020'], ['2', '0', '0'])
(['2012', '2020'], [])
(['2012', '2020'], ['2', '0', '0'])


,0,1,2,3,4,5,6
0,table 13.11.15 Oecd asia Oceania: electricity ...,2012,2020,2025,2030,2040,2050
1,power plants,"1,780","1,935","1,961","2,112","2,454","2,838"
2,hard coal (& non-renewable waste),515,494,408,241,8,0
3,lignite,144,110,92,50,5,0
4,gas,570,452,383,370,267,128
5,of which from H2,0,0,0,0,27,128
6,oil,194,120,72,29,13,0
7,diesel,7,3,3,3,1,0
8,nuclear,166,163,50,0,0,0
9,biomass (& renewable waste),35,74,91,88,59,48


In [751]:
table2 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (410.0, left, 409 + 194, middle))
t2head = table2.columns
if config == 0:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 1:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 2:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 3:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 4:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 5:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 6:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
if config == 7:
    table2 = cleanOverlap(table2)
    table2 = replaceHeader(t2head, table2)
table2

,0,1,2,3,4,5,6
0,table 13.11.16 Oecd asia Oceania: final energy...,2012,2020,2025,2030,2040,2050
1,road,"5,244","4,912","4,276","3,421","2,156","1,521"
2,fossil fuels,"5,167","4,628","3,600","2,325",590,0
3,biofuels,27,146,322,319,178,51
4,synfuels,0,47,34,28,126,129
5,natural gas,51,84,88,75,24,0
6,hydrogen,0,10,52,214,460,579
7,electricity,0,44,214,487,903,890
8,rail,149,157,160,191,231,285
9,fossil fuels,59,41,25,15,4,0


In [752]:
table3 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (596, left, 596 + 183, middle))
t3head = table3.columns
if config == 0:
    #table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 1:
    table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 2:
    #table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 3:
    #table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 4:
    #table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 5:
    #table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 6:
    table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
if config == 7:
    table3 = cleanOverlap(table3)
    table3 = replaceHeader(t3head, table3)
table3

(['2012'], ['1', '172'])
(['2012'], ['2', '30'])
(['2012'], ['3', '7,573'])
(['2012'], ['2', '30'])


,0,1,2,3,4,5,6
0,table 13.11.17 Oecd asia Oceania: heat supply ...,2012,2020,2025,2030,2040,2050
1,district heating plants,153,141,154,192,193,202
2,fossil fuels,140,111,112,108,44,0
3,biomass,12,29,39,57,91,137
4,solar collectors,0,0,1,20,43,45
5,geothermal,0,0,2,8,15,21
6,heat from chp1,172,302,473,611,986,"1,523"
7,fossil fuels,165,223,284,274,201,0
8,biomass,7,43,125,238,543,911
9,geothermal,1,32,60,88,195,418


In [753]:
table4 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (53.02, 22 + 274.96, 53.02 + 145, 540))
table4

,table 13.11.15 Oecd asia Oceania: electricity generation twh/a,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,table 13.11.18: Oecd asia Oceania: installed capacity gw,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2012 2020,2025,2030,2040,2050,2012,2020,2025,2030,2040,2050
1,"power plants 1,780 1,935","1,961","2,112","2,454","2,838",total generation 458,612,752,897,"1,120","1,376"
2,hard coal (& non-renewable waste) 515 494,408,241,8,0,fossil 297,319,297,252,130,0
3,lignite 144 110,92,50,5,0,hard coal (& non-renewable waste) 77,82,70,50,2,0
4,gas 570 452,383,370,267,128,lignite 21,19,16,9,1,0
5,of which from H2 0 0,0,0,27,128,gas (w/o h2) 140,161,171,172,116,0
6,oil 194 120,72,29,13,0,oil 57,56,38,20,10,0
7,diesel 7 3,3,3,1,0,diesel 2,1,2,2,0,0
8,nuclear 166 163,50,0,0,0,nuclear 68,24,7,0,0,0
9,biomass (& renewable waste) 35 74,91,88,59,48,"hydrogen (fuel cells, gas power plants, gas ch...",0,0,0,14,123


In [754]:
#config = 3
if config == 0:
    table4 = table4.iloc[:, -7:]
    t4head = table4.columns
    #table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 1:
    table4 = table4.iloc[:, 5:]
    t4head = table4.columns
    table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 2:
    table4 = table4.iloc[:, -6:]
    t4head = table4.columns[-6:]
    table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 3:
    table4 = table4.iloc[:, -7:]
    t4head = table4.columns[-7:]
    #table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 4:
    table4 = table4.iloc[:, -7:]
    t4head = table4.columns[-7:]
    table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 5:
    table4 = table4.iloc[:, -6:]
    t4head = table4.columns[-6:]
    table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 6:
    table4 = table4.iloc[:, -5:]
    t4head = table4.columns[-5:]
    table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
if config == 7:
    table4 = table4.iloc[:, -6:]
    t4head = table4.columns[-6:]
    table4 = cleanOverlap(table4)
    table4 = replaceHeader(t4head, table4)
table4

(['2012'], ['2', '140'])


,0,1,2,3,4,5,6
0,table 13.11.18: Oecd asia Oceania: installed c...,2012,2020,2025,2030,2040,2050
1,total generation,458,612,752,897,"1,120","1,376"
2,fossil,297,319,297,252,130,0
3,hard coal (& non-renewable waste),77,82,70,50,2,0
4,lignite,21,19,16,9,1,0
5,gas (w/o h2),140,161,171,172,116,0
6,oil,57,56,38,20,10,0
7,diesel,2,1,2,2,0,0
8,nuclear,68,24,7,0,0,0
9,"hydrogen (fuel cells, gas power plants, gas chp)",0,0,0,0,14,123


In [755]:
table5 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (200, middle, 200.5 + 275, right))
t5head = table5.columns

if config == 0:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 1:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 2:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 3:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 4:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 5:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 6:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
if config == 7:
    table5 = cleanOverlap(table5)
    table5 = replaceHeader(t5head, table5)
table5

(['2012', '2020'], ['1', '20,108', '19,882'])


,0,1,2,3,4,5,6
0,table 13.11.19: Oecd asia Oceania: final energ...,2012,2020,2025,2030,2040,2050
1,total (incl. non-energy use),"23,756","22,985","21,379","19,924","16,742","13,651"
2,total energy use1,"20,108","19,882","18,570","17,211","14,265","11,407"
3,transport,"5,881","5,597","4,910","4,046","2,852","2,221"
4,oil products,"5,698","5,127","4,022","2,671",811,0
5,natural gas,67,97,98,82,20,0
6,biofuels,27,156,353,382,247,132
7,synfuels,0,50,37,34,175,331
8,electricity,90,157,347,663,"1,140","1,178"
9,RES electricity,9,47,163,432,994,"1,178"


In [756]:
table6 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (495, middle, 485 + 190, right))

In [757]:
if config ==0:
    t6head = table6.iloc[0]
    table6 = replaceHeader(t6head, table6)
    table6 = topCombine(table6)
if config == 1:
    t6head = table6.iloc[0]
    table6 = cleanOverlap(table6, year_row = 1, drop = True)
    table6 = replaceHeader(t6head, table6)
    #table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
if config == 2:
    t6head = table6.iloc[0]
    #table6 = cleanOverlap(table6, year_row = 1)
    table6 = replaceHeader(t6head, table6)
    table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
if config == 3:
    t6head = table6.iloc[0]
    #table6 = cleanOverlap(table6, year_row = 1)
    table6 = replaceHeader(t6head, table6)
    table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
if config == 4:
    t6head = table6.iloc[0]
    #table6 = cleanOverlap(table6, year_row = 1)
    table6 = replaceHeader(t6head, table6)
    table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
if config == 5:
    t6head = table6.iloc[0]
    #table6 = cleanOverlap(table6, year_row = 1)
    table6 = replaceHeader(t6head, table6)
    table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
if config == 6:
    t6head = table6.iloc[0]
    #table6 = cleanOverlap(table6, year_row = 1)
    table6 = replaceHeader(t6head, table6)
    table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
if config == 7:
    t6head = table6.iloc[0]
    table6 = cleanOverlap(table6, year_row = 1, drop = True)
    #table6 = replaceHeader(t6head, table6)
    #table6 = topCombine(table6)
    table6.iloc[22,1] = table6.iloc[22,2]
    table6 = table6.dropna(axis = 1, thresh = 7)
    table6 = replaceHeader(t6head, table6)
table6

(['2012'], ['2', '1,043'])
(['2012'], [])
(['2012'], ['2', '697'])
(['2012'], ['2', '564'])
(['2012'], ['2', '2,214'])
(['2012'], ['1990', '1,566', '141%'])
(['2012'], ['1', '319'])
(['2012'], ['1', '258'])
(['2012'], ['2', '1,016'])
(['2012'], ['3', '181'])
(['2012'], ['2', '10.9'])


,0,1,2,3,4,5,6
0,table 13.11.20 Oecd asia Oceania: cO2 emission...,2012,2020,2025,2030,2040,2050
1,condensation power plants,998,810,640,415,113,0
2,hard coal (& non-renewable waste),440,404,332,193,6,0
3,lignite,171,122,99,53,5,0
4,gas,254,196,154,145,92,0
5,oil,127,86,53,22,10,0
6,diesel,5,2,2,2,0,0
7,combined heat and power plants,45,54,64,59,37,0
8,hard coal (& non-renewable waste),9,11,14,11,3,0
9,lignite,3,6,5,0,0,0


In [758]:
table7 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = ex_page, area = (685, middle, 685 + 120, right))
if config == 0:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)

if config == 1:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True)
    table7 = replaceHeader(t7head, table7)
    #table7 = topCombine(table7)
    
if config == 2:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)
    #table7 = topCombine(table7)
if config == 3:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)
    #table7 = topCombine(table7)
if config == 4:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)
    #table7 = topCombine(table7)
if config == 5:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)
    #table7 = topCombine(table7)
if config == 6:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)
    #table7 = topCombine(table7)
if config == 7:
    t7head = table7.iloc[0]
    table7 = cleanOverlap(table7, year_row = 1, column = 1, text_issue=False)
    table7 = cleanOverlap(table7, year_row = 1, text_issue=True, drop = True)
    table7 = replaceHeader(t7head, table7)
table7


,0,1,2,3,4,5,6
0,table 13.11.21: Oecd asia Oceania: primary ene...,2012,2020,2025,2030,2040,2050
1,total,"35,801","34,449","31,081","28,680","24,404","20,898"
2,fossil,"32,365","27,306","22,486","17,061","8,073","2,246"
3,hard coal (& non-renewable waste),"8,092","6,863","5,505","3,172",380,20
4,lignite,"1,592","1,133",915,466,46,0
5,natural gas,"7,667","6,457","5,889","5,853","3,611",46
6,crude oil,"15,014","12,853","10,177","7,570","4,036","2,180"
7,nuclear,"1,814","1,778",545,0,0,0
8,renewables,"1,622","5,365","8,050","11,619","16,331","18,652"
9,hydro,419,539,558,571,586,647


In [759]:
page = pd.concat([table1, table2, table3, table4, table5, table6, table7], axis=0)

In [760]:
page

,0,1,2,3,4,5,6
0,table 13.11.15 Oecd asia Oceania: electricity ...,2012,2020,2025,2030,2040,2050
1,power plants,"1,780","1,935","1,961","2,112","2,454","2,838"
2,hard coal (& non-renewable waste),515,494,408,241,8,0
3,lignite,144,110,92,50,5,0
4,gas,570,452,383,370,267,128
5,of which from H2,0,0,0,0,27,128
6,oil,194,120,72,29,13,0
7,diesel,7,3,3,3,1,0
8,nuclear,166,163,50,0,0,0
9,biomass (& renewable waste),35,74,91,88,59,48


## Financial Pages

In [777]:
np.arange(14, 55, 4)

array([14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54])

In [889]:
fin_page = 42
config = 3
table22 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = fin_page, area = (62, 21.97, 62 + 286, 21+ 274.96), columns = [21+274.96])
table22_title = table22.columns

if config ==0 :
    table22 = table22.iloc[:, :5].dropna(axis = 0, thresh = 3)
    table22 = cleanOverlap(table22, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table22_title, table22)
if config ==1:
    table22 = table22.iloc[:, :4].dropna(axis = 0, thresh = 3)
    table22 = cleanOverlap(table22, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table22_title, table22)
if config ==2 :
    table22 = table22.iloc[:, :5].dropna(axis = 0, thresh = 3)
    table22 = cleanOverlap(table22, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table22_title, table22)
if config ==3 :
    table22 = table22.iloc[:, :6].dropna(axis = 0, thresh = 3)
    table22 = cleanOverlap(table22, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table22_title, table22)
if config ==4 :
    table22 = table22.iloc[:, :4].dropna(axis = 0, thresh = 3)
    table22 = cleanOverlap(table22, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table22_title, table22)
table22

,0,1,2,3,4,5,6,7
0,table 13.8.22: india: investments in electrici...,2012-,2021-,2031-,2041-,2012-,unit,annual average
1,ref,2020,2030,2040,2050,2050,NaN,2012-2050
2,fossil (w/o chp) billion $,215.3,354.3,418.6,487.1,"1,475.3",billion $/a,37.8
3,nuclear billion $,18.5,37.8,44.6,51.0,151.9,billion $/a,3.9
4,chp (fossil + renewable) billion $,0.0,0.0,0.0,0.0,0.0,billion $/a,0.0
5,renewables (w/o chp) billion $,109.8,153.0,181.7,187.3,631.7,billion $/a,16.2
6,total billion $,343.6,545.1,644.9,725.4,"2,259.0",billion $/a,57.9
7,conventional (fossil & nuclear) billion $,233.8,392.1,463.2,538.1,"1,627.3",billion $/a,41.7
8,renewables billion $,109.8,153.0,181.7,187.3,631.7,billion $/a,16.2
9,biomass billion $,10.2,13.3,17.9,19.5,60.8,billion $/a,1.6


In [890]:
table23 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = fin_page, area = (55, 300, 55 + 170, 300 + 285))
table23_title = table23.columns


if config ==0:
    table23_starting = 6
    table23 = table23.iloc[:, table23_starting:].dropna(axis = 0, thresh = 3)
    table23 = cleanOverlap(table23, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table23_title[table23_starting:], table23)
if config ==1:
    table23_starting = 4
    table23 = table23.iloc[:, table23_starting:].dropna(axis = 0, thresh = 3)
    table23 = cleanOverlap(table23, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table23_title[table23_starting:], table23)
if config ==2:
    table23_starting = 5
    table23 = table23.iloc[:, table23_starting:].dropna(axis = 0, thresh = 3)
    table23 = cleanOverlap(table23, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table23_title[table23_starting:], table23)
if config ==3:
    table23_starting = 6
    table23 = table23.iloc[:, table23_starting:].dropna(axis = 0, thresh = 3)
    table23 = cleanOverlap(table23, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table23_title[table23_starting:], table23)
if config ==4:
    table23_starting = 4
    table23 = table23.iloc[:, table23_starting:].dropna(axis = 0, thresh = 3)
    table23 = cleanOverlap(table23, financial = True).dropna(axis =1, thresh=7)
    replaceHeader(table23_title[table23_starting:], table23)
table23

,0,1,2,3,4,5,6,7
0,table 13.8.23: india: investments in renewable...,2012-,2021-,2031-,2041-,2012-,unit,annual average
1,ref,2020,2030,2040,2050,2050,NaN,2012-2050
2,heat pumps billion $,0.1,0.2,0.2,0.3,0.8,billion $/a,0.0
3,deep geothermal billion $,0.0,0.0,0.0,0.0,0.0,billion $/a,0.0
4,solar thermal billion $,3.2,4.0,6.7,8.2,22.1,billion $/a,0.6
5,biomass billion $,170.0,54.8,38.0,46.7,309.5,billion $/a,7.9
6,total billion $,173.3,59.0,44.9,55.2,332.5,billion $/a,8.5
7,heat pumps billion $,1.1,62.9,71.8,134.0,269.9,billion $/a,6.9
8,deep geothermal billion $,0.0,25.2,48.5,100.5,174.2,billion $/a,4.5
9,solar thermal billion $,5.2,105.5,144.0,173.9,428.5,billion $/a,11.0


In [891]:
table24 = tabula.read_pdf('Energy-Revolution-2015-Full.pdf', pages = fin_page, area = (190, middle, 190 + 164, right), columns = [380, 406, 435, 465, 490, 520, 545])
if config ==0:
    t24header = table24.iloc[8, -9:]
    table24 = table24.iloc[8:,-9:].dropna(axis =1, thresh = 7).dropna(axis = 0, thresh = 3).reset_index(drop = True)
    #replaceHeader(table1.columns, table1)
    #table24 = cleanOverlap(table24, financial = True)
    fillBlanks(table24, year_row = 1)
    replaceHeader(t24header, table24)
    topCombine(table24)
if config ==1:
    t24header = table24.iloc[8, -9:]
    table24 = table24.iloc[8:,-9:].dropna(axis = 0, thresh = 3).reset_index(drop = True)#.dropna(axis =1, thresh = 7)
    #table24 = cleanOverlap(table24, tex)
    table24 = fillBlanks(table24, year_row = 2, columns = 1)
    table24 = fillBlanks(table24, year_row = 1, columns = 0).dropna(axis =1, thresh = 7)
    table24 = replaceHeader(t24header, table24)
    table24 = topCombine(table24)
if config ==2:
    t24header = table24.iloc[8, -9:]
    table24 = table24.iloc[8:,-9:].dropna(axis = 0, thresh = 3).reset_index(drop = True)#.dropna(axis =1, thresh = 7)
    #table24 = cleanOverlap(table24, tex)
    #table24 = fillBlanks(table24, year_row = 2, columns = 1)
    table24 = fillBlanks(table24, year_row = 1, columns = 0).dropna(axis =1, thresh = 7)
    table24 = replaceHeader(t24header, table24)
    table24 = topCombine(table24)
if config ==3:
    t24header = table24.iloc[8, -9:]
    table24 = table24.iloc[8:,-9:].dropna(axis = 0, thresh = 3).reset_index(drop = True)#.dropna(axis =1, thresh = 7)
    #table24 = cleanOverlap(table24, tex)
    #table24 = fillBlanks(table24, year_row = 2, columns = 1)
    table24 = fillBlanks(table24, year_row = 1, columns = 0).dropna(axis =1, thresh = 7)
    table24 = replaceHeader(t24header, table24)
    table24 = topCombine(table24)
if config ==4:
    t24header = table24.iloc[8, -9:]
    table24 = table24.iloc[8:,-9:].dropna(axis = 0, thresh = 3).reset_index(drop = True)#.dropna(axis =1, thresh = 7)
    #table24 = cleanOverlap(table24, tex)
    table24 = fillBlanks(table24, year_row = 1, columns = 5)
    table24 = fillBlanks(table24, year_row = 1, columns = 0).drop(columns = "1,792.7")
    table24 = replaceHeader(t24header, table24)
    table24 = topCombine(table24)

table24

,0,1,2,3,4,5,6,7
0,table 13.8.24: india: total employment in the ...,2015,2020,2025,2030,2020,2025,2030
1,construction and installation,"1,226","1,622","1,427","1,204","1,459","2,532","2,283"
2,manufacturing,628,819,719,598,"1,691","2,627","2,395"
3,operations and maintenance,414,560,654,621,780,"1,239","1,625"
4,fuel supply (domestic),"2,265","2,323","2,211","1,826","2,304","2,106","1,540"
5,coal and gas export,-,-,-,NaN,-,-,-
6,solar and geothermal heat,20,20,23,22,50,466,733
7,total jobs (thousands),"4,554","5,344","5,035","4,271","6,284","8,969","8,576"
8,coal,"2,252","2,769","2,552","2,324",839,618,384
9,"gas, oil & diesel",224,299,362,343,319,437,357


In [892]:
financial_page = pd.concat([table22, table23, table24], axis=0)

In [893]:
financial_page

,0,1,2,3,4,5,6,7
0,table 13.8.22: india: investments in electrici...,2012-,2021-,2031-,2041-,2012-,unit,annual average
1,ref,2020,2030,2040,2050,2050,NaN,2012-2050
2,fossil (w/o chp) billion $,215.3,354.3,418.6,487.1,"1,475.3",billion $/a,37.8
3,nuclear billion $,18.5,37.8,44.6,51.0,151.9,billion $/a,3.9
4,chp (fossil + renewable) billion $,0.0,0.0,0.0,0.0,0.0,billion $/a,0.0
5,renewables (w/o chp) billion $,109.8,153.0,181.7,187.3,631.7,billion $/a,16.2
6,total billion $,343.6,545.1,644.9,725.4,"2,259.0",billion $/a,57.9
7,conventional (fossil & nuclear) billion $,233.8,392.1,463.2,538.1,"1,627.3",billion $/a,41.7
8,renewables billion $,109.8,153.0,181.7,187.3,631.7,billion $/a,16.2
9,biomass billion $,10.2,13.3,17.9,19.5,60.8,billion $/a,1.6


## Storage

In [167]:
#initialize
storage = page.copy()

In [776]:
fin_storage = financial_page.copy()

#### adding

In [ ]:
11,12,13, c.0
15,16,17, c.1
19,20,21, c.2
23,24,25, c.2
27,28,29, c.3
31,32, c.4
33,35 c.2
36,37 c.6
39,40,41 c.0
43,44,45 c.2/3
47,48,49 c.0
51,52,53 c.7

In [761]:
storage = pd.concat([storage,page], axis=0)

In [ ]:
storage.to_csv("storage54.csv")

#### financial csvs

In [ ]:
14 c.0
18 c.1
22, 26 c.2
30 c.3
34 c.2
38 c.4
42 c.3

In [894]:
fin_storage = pd.concat([fin_storage,financial_page], axis=0)

In [762]:
fin_storage.to_csv("fin_storage.csv")